In [31]:
import pandas as pd
import requests
import numpy as np

In [32]:
url = "https://api.ipma.pt/open-data/distrits-islands.json" # this API makes the conection between "globalIdLocal" and "local" (which is the city/district)

response = requests.get(url=url)
# response.status_code  #insert an if condition here == 200 to check if the status is OK

In [33]:
# response.json() #getting the content of the json

# dividing the dictionary in column "data" to obtain the information about location

ipma_loc =  pd.DataFrame(response.json()) #turns "response" into a dataframe
ipma_data = pd.json_normalize(ipma_loc["data"]) # unfolds the column "data" which had a dictionary. gets all columns: idRegiao idAreaAviso	idConcelho	globalIdLocal	latitude	idDistrito	local	longitude
ipma_df_loc = ipma_data[["globalIdLocal", "local"]] # selecting the two columns to match the other API with the weather
# matching "globalIdLocal" with "local" to do a forloop in the {globalIdLocal} "https://api.ipma.pt/open-data/forecast/meteorology/cities/daily/{globalIdLocal}.json" API link

In [34]:
list_globalidloc = list(ipma_df_loc["globalIdLocal"].values) #converting "globalIdLocal" into a list. Thhis will be useful for the forloop to all the cities

In [35]:
a = list_globalidloc[0] #testing with just one value if the other API returns the correct link
url2 = "http://api.ipma.pt/open-data/forecast/meteorology/cities/daily/"
response2 = requests.get(url = url2 + str(a)+".json")
# response2.status_code #insert an if condition here == 200 to check if the status is OK
# response2.url #the statuscode returned 200, and the link is correct with the value from the list of the other API


In [36]:
url2 = "http://api.ipma.pt/open-data/forecast/meteorology/cities/daily/"

ipma_dict = {} # creating a dictionary that compiles every single globalIdLocal to generate as much links as local id's
for idloc in list_globalidloc:
    url =  url2 + str(idloc) + ".json" # adding the extention to the main API
    response = requests.get(url = url) # requesting every single link
    ipma_dict[idloc] = response.json() # getting a .json from every single loop
    
ipma_dict #all compiled information in dictionary format
ipma_df = pd.DataFrame(ipma_dict).T #converting the dict to a data frame, where the column "data" needs to be decomposed 
ipma_df.reset_index(inplace=True)

In [37]:
to_bread_df = pd.json_normalize(ipma_df["data"]) # creating a new variable that has JUST the column "data" to decomposed
# creating a for loop because the "to_bread_df" decomposed has 5 columns to decompose themselves
for i in range(len(to_bread_df.columns)-2): #"-2" because the forecast is for 5 days and we just need 3
    temp = pd.json_normalize(to_bread_df[i]) # decomposing every single column with the loop
    ipma_df = pd.concat([ipma_df, temp], axis=1) #getting the columns of the initial dataframe, with the decomposed ones

ipma_df.head(2)


,index,owner,country,data,globalIdLocal,dataUpdate,precipitaProb,tMin,tMax,predWindDir,...,precipitaProb,tMin,tMax,predWindDir,idWeatherType,classWindSpeed,longitude,forecastDate,classPrecInt,latitude
0,1010500,IPMA,PT,"[{'precipitaProb': '100.0', 'tMin': '15.5', 't...",1010500,2022-10-20T15:31:04,100.0,15.5,20.7,S,...,100.0,16.1,21.6,S,6,2,-8.6535,2022-10-22,2.0,40.6413
1,1020500,IPMA,PT,"[{'precipitaProb': '100.0', 'tMin': '15.8', 't...",1020500,2022-10-20T15:31:04,100.0,15.8,19.3,S,...,97.0,15.7,20.8,S,6,2,-7.8700,2022-10-22,2.0,38.0200


In [38]:
#clearing the columns that aren't needed
ipma_clear = ipma_df.drop(columns=["country","data","predWindDir","idWeatherType","classWindSpeed","classPrecInt","dataUpdate","longitude","latitude","index","owner"]) 
# filtering the columns by day
hoje = ipma_clear.iloc[:,:5] 
amanha = ipma_clear.iloc[:,[0,5,6,7,8]]
depois_de_amanha = ipma_clear.iloc[:,[0,9,10,11,12]]
vertical_concat = pd.concat([hoje, amanha, depois_de_amanha], axis=0) # getting all the columns together to show the result
all_columns = pd.merge(vertical_concat, ipma_df_loc) # merging the column "location" of the dataframe "ipma_df" from the firs API.

In [ ]:
def get_dist(val):
    return a.loc[(a["FREGUESIA"] == val) | (a["Concelho"] == val), "Distrito"].values[0]
get_dist("PORTO DE MOS")

In [39]:
all_columns

,globalIdLocal,precipitaProb,tMin,tMax,forecastDate,local
0,1010500,100.0,15.5,20.7,2022-10-20,Aveiro
1,1010500,100.0,16.4,20.5,2022-10-21,Aveiro
2,1010500,100.0,16.1,21.6,2022-10-22,Aveiro
3,1020500,100.0,15.8,19.3,2022-10-20,Beja
4,1020500,98.0,15.7,18.8,2022-10-21,Beja
...,...,...,...,...,...,...
100,3480200,7.0,15.6,21.4,2022-10-21,Santa Cruz das Flores
101,3480200,6.0,16.1,23.0,2022-10-22,Santa Cruz das Flores
102,3490100,89.0,17.5,20.8,2022-10-20,Vila do Corvo
103,3490100,25.0,16.8,21.6,2022-10-21,Vila do Corvo


In [40]:
a= pd.read_excel("list_freguesias.xlsx")
a

,Distrito,Concelho,FREGUESIA,CÓDIGO
0,AVEIRO,AGUEDA,AGADÃO,10101
1,AVEIRO,AGUEDA,AGUADA DE BAIXO,10102
2,AVEIRO,AGUEDA,AGUADA DE CIMA,10103
3,AVEIRO,AGUEDA,AGUEDA,10104
4,AVEIRO,AGUEDA,BARRO,10105
...,...,...,...,...
4284,FUNCHAL,SANTANA,S. ROQUE DO FAIAL,221005
4285,FUNCHAL,SANTANA,ILHA,221006
4286,FUNCHAL,S. VICENTE,BOA VENTURA,221101
4287,FUNCHAL,S. VICENTE,PONTA DELGADA,221102


In [54]:
def get_dist(val):
    capital = a.loc[(a["FREGUESIA"] == val) | (a["Concelho"] == val), "Distrito"].values[0]
    return capital

get_dist("VENDAS NOVAS")


'EVORA'